# OPEN METEO API

In [ ]:
#Install
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas!pip install python-dotenv

In [ ]:
#Import Dependencies
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [ ]:
#Setup the Open-Meteo API Client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [ ]:
#Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": [36.154, 40.7128, 34.0549],
	"longitude": [95.9928, 74.006, 118.2426],
	"start_date": "2014-01-01",
	"end_date": "2023-12-31",
	"hourly": "relative_humidity_2m",
	"daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum"],
	"temperature_unit": "fahrenheit",
	"precipitation_unit": "inch",
	"timezone": "America/Chicago"
}
responses = openmeteo.weather_api(url, params=params)

In [ ]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

In [ ]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_relative_humidity_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

In [ ]:
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["precipitation_sum"] = daily_precipitation_sum

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

# EPA API CONCATED

In [ ]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv('EPA_API_KEY')
email_key = os.getenv('MY_EMAIL')


def fetch_aqi_data(year, api_key):
    url = 'https://aqs.epa.gov/data/api/annualData/byCBSA'
    params = {
        'email': email_key,
        'key': api_key,
        'param': '88101',  # Parameter code for PM2.5
        'bdate': f'{year}0101',
        'edate': f'{year}1231',
        'cbsa': '35620'  # CBSA code for the desired area
    }
    response = requests.get(url, params=params)
    data = response.json()
    return pd.DataFrame(data['Data'])

# Define the range of years
years = range(2014, 2023)  # Adjust this range as needed

# Fetch and concatenate data for each year
all_data = pd.DataFrame()
for year in years:
    yearly_data = fetch_aqi_data(year, api_key)
    all_data = pd.concat([all_data, yearly_data])

# Reset the index after concatenation
all_data.reset_index(drop=True, inplace=True)

# Convert the correct date column to datetime format
all_data['first_max_datetime'] = pd.to_datetime(all_data['first_max_datetime'])

# Set the correct date column as the index
all_data.set_index('first_max_datetime', inplace=True)

# View the combined DataFrame
print(all_data.head())
print(all_data.tail())

# DATA CONCATED 

In [ ]:
#Code to Combine Data (Under Construction)
Error404